/home/gustavo# Feature engineering
Nesse notebook vamos tratar os dados dos jogadores e criar features novas com bases nos dados que temos

In [25]:
from database.bq_connection import execute
from sklearn.preprocessing import LabelBinarizer
from datetime import date, datetime
import pandas as pd
import os

In [26]:
def get_project_root_path():
    project_directory = "fifa22-players-analysis"
    separated_path = os.getcwd().split(os.sep)
    project_directory_index = separated_path.index(project_directory) + 1
    return os.sep.join(separated_path[0:project_directory_index])

In [27]:
query = """SELECT * FROM `puc-sp.fifa.player`"""

file_key_path = os.sep.join([get_project_root_path(), "database", "filekey.key"])
data = execute(query, file_key_path)

print(data.shape)
data

(19239, 90)


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,GK,76,76,NaN,NaN,1987-01-27,...,29+2,29+2,29+2,27+2,27+2,28+2,28+2,28+2,27+2,75+1
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,GK,76,76,NaN,NaN,1987-10-23,...,37+2,37+2,37+2,31+2,31+2,35+2,35+2,35+2,31+2,75+1
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,GK,74,74,NaN,NaN,1988-03-17,...,22+2,22+2,22+2,19+2,18+2,22+2,22+2,22+2,18+2,73+1
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,GK,72,72,NaN,NaN,1993-03-21,...,27+2,27+2,27+2,26+2,26+2,27+2,27+2,27+2,26+2,71+1
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,GK,72,82,NaN,NaN,1999-02-25,...,24+2,24+2,24+2,19+2,19+2,25+2,25+2,25+2,19+2,71+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,262593,https://sofifa.com/player/262593/moritz-seiffe...,M. Seiffert,Moritz Seiffert,LW,58,66,450000.0,600.0,2000-11-04,...,40+2,40+2,40+2,44+2,42+2,37+2,37+2,37+2,42+2,14+2
19235,262614,https://sofifa.com/player/262614/till-muschkow...,T. Muschkowski,Till Muschkowski,RW,55,65,275000.0,550.0,2001-05-06,...,44+2,44+2,44+2,47+2,45+2,39+2,39+2,39+2,45+2,11+2
19236,262755,https://sofifa.com/player/262755/shalva-ogbaid...,S. Ogbaidze,Shalva Ogbaidze,ST,55,65,275000.0,500.0,2002-08-01,...,39+2,39+2,39+2,40+2,38+2,36+2,36+2,36+2,38+2,14+2
19237,262756,https://sofifa.com/player/262756/matteo-gumane...,M. Gumaneh,Matteo Gumaneh,ST,55,67,300000.0,500.0,2002-09-05,...,37+2,37+2,37+2,40+2,38+2,34+2,34+2,34+2,38+2,15+2


Nesse dataset temos apenas `dob` (data de nascimento) dos jogadores, vamos adicionar uma coluna com as idades dos jogadores

In [28]:
current_date = datetime.today()
data.dob = pd.to_datetime(data.dob)

def age(dob):
    today = date.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

data["age"] = data.dob.apply(age)
data[["dob", "age"]].head()

,dob,age
0,1987-01-27,35
1,1987-10-23,34
2,1988-03-17,34
3,1993-03-21,29
4,1999-02-25,23


Outra tratativa a ser feita é transformar cada `player_traits` e cada `player_tags` diferente em uma coluna boleana

In [29]:
data[["player_traits", "player_tags", "player_positions"]]

,player_traits,player_tags,player_positions
0,Team Player,None,GK
1,"Leadership, Comes For Crosses",None,GK
2,None,None,GK
3,"GK Long Throw, Cautious With Crosses",None,GK
4,None,None,GK
...,...,...,...
19234,None,None,LW
19235,None,None,RW
19236,None,None,ST
19237,None,None,ST


In [30]:
player_traits_dummies = data.player_traits.str.get_dummies(sep=', ')
player_tags_dummies = data.player_tags.str.get_dummies(sep=', ')
player_positions_dummy = data.player_positions.str.get_dummies(sep=', ')

processed_data = pd.concat([data, player_traits_dummies, player_tags_dummies, player_positions_dummy], axis=1)
processed_data.drop(columns=["player_traits", "player_tags", "player_positions"], inplace=True)
processed_data.head()

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,...,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,...,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,...,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,...,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,...,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,...,1,0,0,0,0,0,0,0,0,0


In [31]:
# Convertendo o tipo das colunas
columns_to_evaluate = "ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk".split(",")
for column_to_evaluate in columns_to_evaluate:
    evaluated_column = processed_data[column_to_evaluate].apply(lambda cell: eval(cell))
    processed_data[column_to_evaluate] = evaluated_column

In [32]:
work_rate_encoder = LabelBinarizer()
body_type_encoder = LabelBinarizer()
preferred_foot_encoder = LabelBinarizer()


processed_data.work_rate = work_rate_encoder.fit_transform(processed_data.work_rate)
processed_data.body_type = body_type_encoder.fit_transform(processed_data.body_type)
processed_data.preferred_foot = preferred_foot_encoder.fit_transform(processed_data.preferred_foot)


processed_data.head()

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,...,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,...,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,...,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,...,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,...,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,...,1,0,0,0,0,0,0,0,0,0


In [33]:
print(f"{work_rate_encoder.classes_=}\n\n{body_type_encoder.classes_=}\n\n{preferred_foot_encoder.classes_=}")

work_rate_encoder.classes_=array(['High/High', 'High/Low', 'High/Medium', 'Low/High', 'Low/Low',
       'Low/Medium', 'Medium/High', 'Medium/Low', 'Medium/Medium'],
      dtype='<U13')

body_type_encoder.classes_=array(['Lean (170-)', 'Lean (170-185)', 'Lean (185+)', 'Normal (170-)',
       'Normal (170-185)', 'Normal (185+)', 'Stocky (170-)',
       'Stocky (170-185)', 'Stocky (185+)', 'Unique'], dtype='<U16')

preferred_foot_encoder.classes_=array(['Left', 'Right'], dtype='<U5')


In [41]:
data_path = os.sep.join([get_project_root_path(), "preprocessing", "data", f"processed_data_{str(date.today())}.csv"])
processed_data.to_csv(data_path, index=False)

In [42]:
data_path

'/home/gustavo/fifa22-players-analysis/preprocessing/data/processed_data_2022-06-13.csv'